In [1]:
#### Import necessary libraries
import pandas as pd
import os
import glob
from pathlib import Path
import numpy as np

Import_List = ['OESHLST180101to180331','OESHLST180401to180630','OESHLST180701to180930','OESHLST180901to181231','OESHLST190101to190331',
              'OESHLST190401to190630','OESHLST190701to190930','OESHLST191001to191231','OESHLST200101to200831','OESHLST200901to201231',
              'OESHLST210101to210430']
for i in Import_List:
    #### Import csv file
    csvfilename = i
    path = "C:\\Users\\user1\\Desktop\\PythonReport\\OE\\Import\\"+csvfilename+".csv"
    all_data = pd.read_csv(path,header=None)
    all_data = all_data.rename({0:'Item',1:'ItemNumber',3:'DocDate',5:'qty',7:'RetailPrice',15:'Discount'},axis=1) #Rename necessary Header


    #### Find keyword and filter SalesPerson by index to add in branch name in new column
    search_SalesPerson = ['Actual']
    SalesPerson_df = all_data[all_data['ItemNumber'].str.contains('|'.join(search_SalesPerson),case=False, na=False)]
    SalesPerson_index = SalesPerson_df.index - 4
    #Find string "Actual"'s previous 4 cell's index in column C

    search_SalesPerson2 = ['Report Total:']
    SalesPerson_df2 = all_data[all_data['Item'].str.contains('|'.join(search_SalesPerson2),case=False, na=False)]
    SalesPerson_index2 = SalesPerson_df2.index - 6
    #Find string "Report Total:"'s previous 6 cell's index in column A

    SalesPerson_eachdf = all_data['ItemNumber'][all_data.index.isin(SalesPerson_index)]
    SalesPerson_eachdf2 = all_data['ItemNumber'][all_data.index.isin(SalesPerson_index2)]
    SalesPerson_eachdf3 = list(SalesPerson_eachdf) + list(SalesPerson_eachdf2)
    SalesPerson_eachdf3 = pd.DataFrame(SalesPerson_eachdf3)
    SalesPerson_eachdf3 = SalesPerson_eachdf3.rename({0:'SalesPerson'},axis=1)

    SalesPerson_df3 = pd.concat([SalesPerson_df[1:],SalesPerson_df2],join='inner')


    ####Define add in new column
    def AddInNewColumn(Each_df,Each_df_ColumnName,Reference_df,Add_to_df,Add_to_df_ColumnName):
        OutputListName =[]
        for i in range(0,len(Each_df.index)):
            data = [Each_df.loc[Each_df.index[i],str(Each_df_ColumnName)]
                   ] * min(abs(Reference_df.index[i] - Reference_df.index[i-1]),Reference_df.index[i])
            data = list(data)
            OutputListName.extend(data)
        Add_to_df[str(Add_to_df_ColumnName)] = pd.Series(OutputListName)

    #### Find keyword and filter Branches by index
    search_Branch = ['Ship To:']
    Branch_df = all_data[all_data[2].str.contains('|'.join(search_Branch),case=False, na=False)]
    Branch_index = Branch_df.index +1
    Branch_df = all_data['Item'][all_data.index.isin(Branch_index)]
    Branch_df = pd.DataFrame(Branch_df)
    #Find string "Ship To:"'s next cell's index in column C

    #### Add in branch name in new column
    AddInNewColumn(Each_df=Branch_df,Each_df_ColumnName='Item',
                   Reference_df=SalesPerson_df3,Add_to_df=all_data,Add_to_df_ColumnName='Branch')


    #### Find keyword and filter DocNumber by index to add in branch name in new column
    search_Docno = ['Expected']
    Docno_df = all_data[all_data['Item'].str.contains('|'.join(search_Docno),case=False, na=False)]
    Docno_index = Docno_df.index +1
    Docno_df = all_data[all_data.index.isin(Docno_index)]
    Docno_df = pd.DataFrame(Docno_df)
    #Find string "Expected"'s next cell's index in column A

    #### Add in DocNumber in new column
    AddInNewColumn(Each_df=Docno_df,Each_df_ColumnName='Item',
                   Reference_df=SalesPerson_df3,Add_to_df=all_data,Add_to_df_ColumnName='DocNumber')

    #### Add in DocDate in new column
    AddInNewColumn(Each_df=Docno_df,Each_df_ColumnName='DocDate',
                   Reference_df=SalesPerson_df3,Add_to_df=all_data,Add_to_df_ColumnName='DocDate')

    #### Add in CompanyCode in new column
    AddInNewColumn(Each_df=Docno_df,Each_df_ColumnName='ItemNumber',
                   Reference_df=SalesPerson_df3,Add_to_df=all_data,Add_to_df_ColumnName='CompanyCode')

    #### Add in Salesperson in new column
    AddInNewColumn(Each_df=SalesPerson_eachdf3,Each_df_ColumnName='SalesPerson',
                   Reference_df=SalesPerson_df3,Add_to_df=all_data,Add_to_df_ColumnName='SalesPerson')


    all_data['Discount_rate'] = np.where(all_data['Discount'].str.startswith("Less Shipment Disc."),all_data['Discount'],np.nan)
    all_data['Discount_rate'] = all_data['Discount_rate'].fillna(method ='backfill')
    all_data['Discount_rate'] = all_data['Discount_rate'].str[19:-1].astype('float')

    #Get address
    all_data['Address1'] = all_data['Item'].shift(-2)
    all_data['Address2'] = all_data['Item'].shift(-3)
    all_data['Address3'] = all_data['Item'].shift(-4)
    all_data['Address4'] = all_data['Item'].shift(-5)
    all_data['Address5'] = all_data['Item'].shift(-6)
    all_data['Address6'] = all_data['Item'].shift(-7)
    all_data['Address1'] = np.where(all_data[2]=='Ship To:',all_data["Address1"],"")
    all_data['Address2'] = np.where(all_data[2]=='Ship To:',all_data["Address2"],"")
    all_data['Address3'] = np.where(all_data[2]=='Ship To:',all_data["Address3"],"")
    all_data['Address4'] = np.where(all_data[2]=='Ship To:',all_data["Address4"],"")
    all_data['Address5'] = np.where(all_data[2]=='Ship To:',all_data["Address5"],"")
    all_data['Address6'] = np.where(all_data[2]=='Ship To:',all_data["Address6"],"")

    #all_data[['Address1','Address2','Address3','Address4','Address5','Address6']] = all_data[['Address1','Address2','Address3','Address4','Address5','Address6']].fillna(method ='ffill')
    all_data[["Address1","Address2","Address3","Address4","Address5","Address6"]] = all_data[["Address1","Address2","Address3","Address4","Address5","Address6"
                                                                                             ]].replace({np.nan:" ","":np.nan})

    all_data['full_address'] = all_data["Address1"] + " " + all_data["Address2"] + " " + all_data["Address3"] + " " + all_data["Address4"]+ " " + all_data["Address5"]+ " " + all_data["Address6"]
    all_data['full_address'] = all_data['full_address'].fillna(method ='ffill')
    #### Find keyword and filter Item Number by index
    search_Itemno = ['Item Number:']
    Itemno_df = all_data[all_data['Item'].str.contains('|'.join(search_Itemno),case=False, na=False)]
    Itemno_index = Itemno_df.index
    Itemno_df = all_data[all_data.index.isin(Itemno_index)]
    #Find string "Expected"'s next cell's index in column A

    Final_df = pd.DataFrame(Itemno_df)
    #Convert all data into data frame

    #### Find keyword and filter Item Number by index
    search_Itemqty = ['GST']
    Itemqty_df = all_data[all_data[2].str.contains('|'.join(search_Itemqty),case=False, na=False)]
    Itemqty_index = Itemqty_df.index
    Itemqty_df = all_data[all_data.index.isin(Itemqty_index)]
    #Find string "GST"'s last cell's index in column A


    Final_df = Final_df.reset_index()
    Final_df = Final_df[{'ItemNumber','DocDate','Branch','DocNumber','CompanyCode','SalesPerson','Discount_rate','full_address'}]
    Itemqty_df = Itemqty_df.reset_index()
    Itemqty_df = Itemqty_df.rename(columns={'ItemNumber':'Product_Code'})
    Itemqty_df = Itemqty_df.rename(columns={'Item':'Tax_Code'})
    Itemqty_df = Itemqty_df[{'Product_Code','Tax_Code','qty','RetailPrice'}]
    Final2_df = pd.concat([Final_df,Itemqty_df],axis=1,sort=False)
    Final2_df['RetailPrice'] = Final2_df['RetailPrice'].str.replace(',','').astype('float')
    Final2_df['qty'] = Final2_df['qty'].str.replace(',','').astype('int')
    Product_df = pd.read_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\Import\\ProductType.csv")
    Tax_df = pd.read_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\Import\\TaxType.csv")
    
    MissingPerson_df = pd.read_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\Import\\MissingSalesPerson.csv")[['DocNumber','MissingSalesPerson']]
    Final2_df = Final2_df.merge(MissingPerson_df,on='DocNumber',how='left',suffixes=('_1',"_2"))
    Final2_df['SalesPerson'] = Final2_df['SalesPerson'].fillna(Final2_df['MissingSalesPerson'])
    
    Manager_df = pd.read_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\Import\\SalesManager.csv")
    Final2_df = Final2_df.merge(Product_df,on='Product_Code',how='left',suffixes=('_1',"_2"))
    Final2_df = Final2_df.merge(Tax_df,on='Tax_Code',how='left',suffixes=('_1',"_2"))
    Final2_df['RetailPrice'] = np.where(Final2_df['Tax_Type']=="INC",Final2_df['RetailPrice']/1.07,Final2_df['RetailPrice'])
    Final2_df = Final2_df.merge(Manager_df,on='SalesPerson',how='left',suffixes=('_1',"_2"))


    Final2_df['RetailPriceAfterDiscount'] = Final2_df.RetailPrice - (Final2_df.RetailPrice * Final2_df.Discount_rate/100)
    Final2_df['Sales'] = Final2_df['qty'] * Final2_df['RetailPriceAfterDiscount']

    Final2_df[['CompanyCode','CompanyName']] = Final2_df['CompanyCode'].str.split('   ',expand=True)
    Final2_df['Doc_Type'] = np.nan
    Final2_df['Doc_Type'] = np.where(Final2_df['DocNumber'].str.startswith('TS0'),"Invoice_Stock_Delivery","Consignment_Stock_Delivery")
    Final2_df['CompanyName'] = np.where(Final2_df['CompanyCode'].str.startswith('NTUC'),"NTUC",
                              np.where(Final2_df['CompanyCode'].str.startswith('POP'),"Popular",
                              np.where(Final2_df['CompanyCode'].str.startswith('MUMM'),"Mummy Market",
                              np.where(Final2_df['CompanyCode'].str.startswith('TOMS'),"Tom & Stefanie",
                              np.where(Final2_df['CompanyCode'].str.startswith('GIAN'),"Giant",
                              np.where(Final2_df['CompanyCode'].str.startswith('SHEN'),"Sheng Siong",Final2_df['CompanyName']))))))


    Final2_df['Branch'] = np.where(Final2_df['Branch']=='KIDDY PALACE PTE LTD',"KIDDY PALACE - KIM CHUAN DRIVE",Final2_df['Branch'])

    #fill postal code for missing data
    Ship_df = pd.read_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\Import\\Shipping_address_amendment.csv")[['full_address','Ship_address']]
    Final2_df = Final2_df.merge(Ship_df,on='full_address',how='left',suffixes=('_1',"_2"))
    Final2_df['Ship_address'] = Final2_df['Ship_address'].fillna(Final2_df['full_address'])
    #Get Postal code only
    Final2_df['Postal_code'] = Final2_df['Ship_address'].str.extract('(\d{6})', expand=False)

    Region_df = pd.read_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\Import\\Singapore_regions.csv")[['Postal Sector (1st 2 digits of 6-digit postal codes)','General Location','Region','Postal District']]
    Region_df = Region_df.rename(columns={'Postal Sector (1st 2 digits of 6-digit postal codes)':'Region_code'})
    Final2_df["Region_code"] = Final2_df["Postal_code"].astype(int) // 10000
    Final2_df = Final2_df.merge(Region_df,on='Region_code',how='left',suffixes=('_1',"_2"))

    Final2_df = Final2_df[['DocDate','Doc_Type','DocNumber','ItemNumber','CompanyCode','CompanyName','Branch','SalesPerson',
                          'Manager','Ship_address','Postal_code','Region_code','General Location','Region','Postal District',
                          'Product_Type','Product_Category','Tax_Type','RetailPrice','Discount_rate',
                          'RetailPriceAfterDiscount','qty','Sales']]

    ### Append to existing report if any
    if os.path.isfile("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\10.ItemizeVisualizeReport\\ItemizeVisualizeReport.csv"):
        Append_data = pd.read_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\10.ItemizeVisualizeReport\\ItemizeVisualizeReport.csv")
        Append_data = pd.DataFrame(Append_data)
        Append_df = pd.concat([Append_data,Final2_df],join='inner')
        Append_df.drop_duplicates(keep='last',inplace=True)
    else:
        Append_df = Final2_df
        
    Append_df.to_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\10.ItemizeVisualizeReport\\ItemizeVisualizeReport.csv",index=False,header=True)

print("Report have been exported.")


C:\Users\user1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Report have been exported.


#### Handling Shipping Address error

In [30]:
Ship_amend_df = Final2_df[Final2_df['Postal_code'].isna()]
Ship_amend_df = Ship_amend_df.drop_duplicates(subset=["full_address"])
Ship_amend_df = Ship_amend_df[['CompanyCode','Branch','full_address']]
Ship_amend_df.to_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\Import\\Shipping_address_amendment.csv",mode='a',index=None,header=False)

In [66]:
Final2_df.to_csv("C:\\Users\\user1\\Desktop\\checking.csv",index=False,header=True)

In [51]:
Test_data = pd.read_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\10.ItemizeVisualizeReport\\ItemizeVisualizeReport.csv")
Test_data[Manager]
# Test_data['SalesPerson'] = np.where(Test_data['MissingSalesPerson']!=np.nan,Test_data['MissingSalesPerson'],Test_data['SalesPerson'])
# Test_data['SalesPerson'] = Test_data['SalesPerson'].fillna(Test_data['MissingSalesPerson'])
Test_data[13309:13314]

C:\Users\user1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,DocDate,Doc_Type,DocNumber,ItemNumber,CompanyCode,CompanyName,Branch,SalesPerson,Manager,Ship_address,...,Postal District,Product_Type,Product_Category,Tax_Type,RetailPrice,Discount_rate,RetailPriceAfterDiscount,qty,Sales,MissingSalesPerson
13309,04/01/2018,Invoice_Stock_Delivery,TS0123787,"TOYS TO SUPPLY PROBOAT ZELOS 36"" TWIN SPEED ...",MONS01,MONSTER DYNAMICS PTE LTD,MONSTER DYNAMICS PTE LTD,JULIA,NaN,32 OLD TOH TUCK ROAD #03-15 I.BIZ CENTRE SE 59...,...,21,TOYS,TOYS,EXC,1027.000000,0.0,1027.000000,1,1027.000000,NaN
13310,04/01/2018,Invoice_Stock_Delivery,TS0123788,116340 DREAMY HIGH CHAIR SEA GR,A&VB01,A & V BEST TRADING,A & V BEST TRADING,WENDY,WENDY,BLK 214 #01-175 BEDOK NORTH STREET 1 SE 460214...,...,16,HIGH CHAIR,COMBI,INC,339.766355,30.0,237.836449,1,237.836449,NaN
13311,04/01/2018,Invoice_Stock_Delivery,TS0123789,115416 MT-700D MIRACLETURN ELEGANT STROLLER*PP,BABY08,BABY GALLERY,BABY GALLERY,WENDY,WENDY,10 ADMIRALTY STREET #05-14 NORTH LINK BUILDING...,...,27,STROLLER,COMBI,INC,559.813084,25.0,419.859813,1,419.859813,NaN
13312,04/01/2018,Invoice_Stock_Delivery,TS0123790,116700 JOY BABY HIGH CHAIR T*YE,BABY08,BABY GALLERY,BABY GALLERY,WENDY,WENDY,10 ADMIRALTY STREET #05-14 NORTH LINK BUILDING...,...,27,HIGH CHAIR,COMBI,INC,372.897196,30.0,261.028037,1,261.028037,NaN
13313,04/01/2018,Invoice_Stock_Delivery,TS0123791,114117 JOYTRIPEG GA BABY CAR SEAT*BK,BABY08,BABY GALLERY,BABY GALLERY,WENDY,WENDY,10 ADMIRALTY STREET #05-14 NORTH LINK BUILDING...,...,27,CAR SEAT,COMBI,INC,298.130841,30.0,208.691589,1,208.691589,NaN


In [28]:
Test_data = pd.read_csv("C:\\Users\\user1\\Desktop\\PythonReport\\OE\\10.ItemizeVisualizeReport\\ItemizeVisualizeReport.csv")
# Test_data.drop_duplicates(keep='last',inplace=True)
Test_data

,DocDate,Doc_Type,DocNumber,ItemNumber,CompanyCode,CompanyName,Branch,SalesPerson,Manager,Ship_address,...,Region,Postal District,Product_Type,Product_Category,Tax_Type,RetailPrice,Discount_rate,RetailPriceAfterDiscount,qty,Sales
0,19/02/2018,Consignment_Stock_Delivery,CON0066223,13028104 CON-250 TS CON TOYS 2.5,NTUCHCBPZ01,NTUC,NTUC-461-HYPER CHANGI BUSINESS,KOO AH WAT,KOO AH WAT,HYPER CHANGI BUSINESS PARK #01-51 & 02-51 8 CH...,...,East,16.0,TOYS,TOYS,EXC,2.50,0.0,2.50,60,150.00
1,19/02/2018,Consignment_Stock_Delivery,CON0066223,13002037 CON-390 TS CON TOYS 3.90,NTUCHCBPZ01,NTUC,NTUC-461-HYPER CHANGI BUSINESS,KOO AH WAT,KOO AH WAT,HYPER CHANGI BUSINESS PARK #01-51 & 02-51 8 CH...,...,East,16.0,TOYS,TOYS,EXC,3.90,0.0,3.90,170,663.00
2,19/02/2018,Consignment_Stock_Delivery,CON0066223,13002038 CON-500 TS CON TOYS 5.00,NTUCHCBPZ01,NTUC,NTUC-461-HYPER CHANGI BUSINESS,KOO AH WAT,KOO AH WAT,HYPER CHANGI BUSINESS PARK #01-51 & 02-51 8 CH...,...,East,16.0,TOYS,TOYS,EXC,5.00,0.0,5.00,288,1440.00
3,19/02/2018,Consignment_Stock_Delivery,CON0066223,13002072 CON-990 TS CON TOYS 9.90,NTUCHCBPZ01,NTUC,NTUC-461-HYPER CHANGI BUSINESS,KOO AH WAT,KOO AH WAT,HYPER CHANGI BUSINESS PARK #01-51 & 02-51 8 CH...,...,East,16.0,TOYS,TOYS,EXC,9.90,0.0,9.90,36,356.40
4,19/02/2018,Consignment_Stock_Delivery,CON0066223,13002075 CON-1390 TS CON TOYS 13.90,NTUCHCBPZ01,NTUC,NTUC-461-HYPER CHANGI BUSINESS,KOO AH WAT,KOO AH WAT,HYPER CHANGI BUSINESS PARK #01-51 & 02-51 8 CH...,...,East,16.0,TOYS,TOYS,EXC,13.90,0.0,13.90,24,333.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265602,31/12/2020,Invoice_Stock_Delivery,TS0133464,114025 COMBI HELLO KITTY FRIENDLY,TOYS01,TOYS'R'US (SINGAPORE) PTE LTD,TOYS'R'US (SINGAPORE) PTE LTD,LILY,WENDY,315 OUTRAM ROAD #14-09 TAN BOON LIAT BUILDING ...,...,Central,3.0,BABY TOYS,COMBI,EXC,34.71,0.0,34.71,3,104.13
265603,31/12/2020,Invoice_Stock_Delivery,TS0133464,115594 701521 COMBI EASY CLEAN HANDY BID YE,TOYS01,TOYS'R'US (SINGAPORE) PTE LTD,TOYS'R'US (SINGAPORE) PTE LTD,LILY,WENDY,315 OUTRAM ROAD #14-09 TAN BOON LIAT BUILDING ...,...,Central,3.0,BABY CARE,COMBI,EXC,9.79,0.0,9.79,1,9.79
265604,31/12/2020,Invoice_Stock_Delivery,TS0133464,115608 TETEO BABY TOOTHBRUSH SET,TOYS01,TOYS'R'US (SINGAPORE) PTE LTD,TOYS'R'US (SINGAPORE) PTE LTD,LILY,WENDY,315 OUTRAM ROAD #14-09 TAN BOON LIAT BUILDING ...,...,Central,3.0,BABY CARE,COMBI,EXC,8.63,0.0,8.63,1,8.63
265605,31/12/2020,Invoice_Stock_Delivery,TS0133464,115609 107360 TETEO BABY TOOTHBRUSH STEP 1,TOYS01,TOYS'R'US (SINGAPORE) PTE LTD,TOYS'R'US (SINGAPORE) PTE LTD,LILY,WENDY,315 OUTRAM ROAD #14-09 TAN BOON LIAT BUILDING ...,...,Central,3.0,BABY CARE,COMBI,EXC,5.16,0.0,5.16,1,5.16
